## Import

In [1]:
import pandas as pd
import numpy as np

In [4]:
df1 = pd.read_csv("Dataset\수도권_방문지_데이터_1.csv")
df2 = pd.read_csv("Dataset\수도권_방문지_데이터_2.csv")

print(df1.head(5))

   VISIT_AREA_ID  TRAVEL_ID  VISIT_ORDER VISIT_AREA_NM VISIT_START_YMD  \
0     2210300006  a_a015688            5        프로방스마을      2022-10-30   
1     2208050001  a_a000164            1        친구/친지집      2022-08-05   
2     2210030002  a_a004293            2       병점역 1호선      2022-10-03   
3     2208200003  a_a000491           12         더현대서울      2022-08-20   
4     2208110007  a_a000172           13        강릉중앙시장      2022-08-11   

  VISIT_END_YMD        ROAD_NM_ADDR           LOTNO_ADDR     X_COORD  \
0    2022-10-30  경기 파주시 탄현면 새오리로 77  경기 파주시 탄현면 성동리 82-1  126.685239   
1    2022-08-05                 NaN       경기도 안산시 단원구 와동         NaN   
2    2022-10-03      경기 화성시 떡전골로 97     경기 화성시 진안동 824-1  127.033200   
3    2022-08-20    서울 영등포구 여의대로 108      서울 영등포구 여의도동 22  126.928446   
4    2022-08-11       강원 강릉시 금성로 21        강원 강릉시 성남동 50  128.898624   

     Y_COORD  ...  POI_NM RESIDENCE_TIME_MIN VISIT_AREA_TYPE_CD REVISIT_YN  \
0  37.790405  ...     NaN               60.0

In [5]:
print(df2.head(5))

   VISIT_AREA_ID  TRAVEL_ID  VISIT_ORDER  VISIT_AREA_NM VISIT_START_YMD  \
0     2210090001  a_a003999            1              집      2022-10-09   
1     2208290001  a_a000605           14      캐니언파크 평택점      2022-08-29   
2     2208050008  a_a000104            8  육회왕자&연어공주 동교점      2022-08-05   
3     2210080006  a_a006203            6            친구집      2022-10-08   
4     2208180004  a_a000198           17           광장시장      2022-08-18   

  VISIT_END_YMD         ROAD_NM_ADDR       LOTNO_ADDR     X_COORD    Y_COORD  \
0    2022-10-09                  NaN              NaN         NaN        NaN   
1    2022-08-29       경기 평택시 비전5로 10  경기 평택시 비전동 1104  127.112541  36.999181   
2    2022-08-05  서울 마포구 어울마당로 133 2층    서울특별시 마포구 동교동  126.924624  37.556223   
3    2022-10-09                  NaN              NaN         NaN        NaN   
4    2022-08-18       서울 종로구 창경궁로 88   서울 종로구 예지동 2-1  126.998949  37.570028   

   ...  POI_NM RESIDENCE_TIME_MIN VISIT_AREA_TYPE_CD REVISIT_YN  \
0

# 필요없는 칼럼 삭제

In [8]:
df1_copy = df1.copy()
df1_copy = df1.drop(['VISIT_AREA_ID','VISIT_ORDER','VISIT_START_YMD','VISIT_END_YMD','X_COORD','Y_COORD','ROAD_NM_CD','LOTNO_CD','POI_ID','POI_NM','VISIT_AREA_TYPE_CD','REVISIT_YN','VISIT_CHC_REASON_CD','LODGING_TYPE_CD','SGG_CD'], axis=1)
print(df1_copy.head())

   TRAVEL_ID VISIT_AREA_NM        ROAD_NM_ADDR           LOTNO_ADDR  \
0  a_a015688        프로방스마을  경기 파주시 탄현면 새오리로 77  경기 파주시 탄현면 성동리 82-1   
1  a_a000164        친구/친지집                 NaN       경기도 안산시 단원구 와동   
2  a_a004293       병점역 1호선      경기 화성시 떡전골로 97     경기 화성시 진안동 824-1   
3  a_a000491         더현대서울    서울 영등포구 여의대로 108      서울 영등포구 여의도동 22   
4  a_a000172        강릉중앙시장       강원 강릉시 금성로 21        강원 강릉시 성남동 50   

   RESIDENCE_TIME_MIN  DGSTFN  REVISIT_INTENTION  RCMDTN_INTENTION  
0                60.0     4.0                4.0               4.0  
1                 NaN     NaN                NaN               NaN  
2                30.0     4.0                5.0               5.0  
3               180.0     5.0                5.0               5.0  
4                30.0     4.0                4.0               4.0  


In [ ]:
removeValue = ['집', '숙소', '친구/친지집']
df_filtered = df1_copy[~df1_copy['VISIT_AREA_NM'].isin(removeValue)]
df_filtered = df1_copy[~df1_copy['VISIT_AREA_NM'].str.contains('아파트')]
